<a href="https://colab.research.google.com/github/IvnGenza/Data-Science/blob/main/HW_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Part 1!**

# ***Given names mining***

In [ ]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd

base_url = 'https://en.wikipedia.org/'            #first part of url for connecting with index of next pages

url =  "https://en.wikipedia.org/wiki/Category:Given_names"


response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")
next_page = soup.findAll('a',string='next page')

names = []


while True:

  h3 = soup.findAll('h3')[10:]

  h3_name_flags = list(filter(lambda x: x.text.isupper() ,h3))
  name_elements = list(map(lambda x:x.find_parent().findAll('a'),h3_name_flags))

  ele_list=[]
  for ele in name_elements:
    ele_list.extend(ele)

  name_elements = ele_list

  plus_names = [i.text for i in name_elements]
  names = names + plus_names

  try:
    href_value = next_page[-1].get('href')             #Getting index of a next page
  except  IndexError:
    break

  next_url = base_url+href_value                    #url of next page

  response = requests.get(next_url)
  soup = BeautifulSoup(response.text, "html.parser")

  next_page = soup.findAll('a',string='next page')

In [ ]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

result_tuples = []
names = iter(names)
while True:

  try:
    name = next(names)
  except Exception as e:
    break

# Construct the API URL
  api_url = f'https://en.wikipedia.org/w/api.php?action=query&titles={name}&prop=pageprops&format=json'

# Make the API request and convert the response to JSON
  response = requests.get(api_url).json()

# Try to Extract the page ID from the JSON response
  try:
    page_id = str(next(iter(response['query']['pages'].values()))['pageprops']['wikibase_item'])

  except Exception as e:
    continue

  url = f"https://www.wikidata.org/wiki/{page_id}"
  response1 = requests.get(url)
  soup = BeautifulSoup(response1.text, "html.parser")
  my_blocks = soup.find('ul',class_='wikibase-sitelinklistview-listview').findAll(recursive=False)

  for my_block in my_blocks:
    s1=my_block.findAll({'span':True})

    my_description = soup.find('span',class_ = 'wikibase-descriptionview-text').text
    my_language = s1[1].get('title')
    my_translate = s1[2].text
    my_lang = s1[3].get('lang')

    result = (name,page_id,my_description,my_language,my_lang,my_translate)
    result_tuples.append(result)



results_df = pd.DataFrame(result_tuples, columns=['Label','WikiData ID','English Description','Language','Wiki Short Lang','Entry'])
results_df.to_csv('names.csv', index=False, encoding='utf-8-sig')

***Part 2!***

# ***Surnames mining:***

In [ ]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
base_url = 'https://en.wikipedia.org/'            #first part of url for connecting with index of next pages

url =  "https://en.wikipedia.org/wiki/Category:Surnames"

session = requests.Session()

response = session.get(url)

soup = BeautifulSoup(response.text, "html.parser")
next_page = soup.findAll('a',string='next page')

names = []


while True:

  h3 = soup.findAll('h3')[13:]
  h3_name_flags = list(filter(lambda x: x.text.isupper() ,h3))
  name_elements = list(map(lambda x:x.find_parent().findAll('a'),tqdm(h3_name_flags)))

  ele_list=[]
  for ele in name_elements:
    ele_list.extend(ele)

  name_elements = ele_list

  plus_names = [i.text for i in name_elements]
  names = names + plus_names

  try:
    href_value = next_page[-1].get('href')             #Getting index of a next page

  except  IndexError:
    break

  next_url = base_url+href_value                    #url of next page

  response = session.get(next_url)
  soup = BeautifulSoup(response.text, "html.parser")

  next_page = soup.findAll('a',string='next page')

In [ ]:
import requests
from requests.adapters import HTTPAdapter
from requests.exceptions import ConnectionError
import json
from bs4 import BeautifulSoup
import pandas as pd
import multiprocessing
from tqdm import tqdm

#########################################################################################################
#########################################################################################################
wiki_adapter = HTTPAdapter(max_retries=99)
session1 = requests.Session()
session1.mount('https://en.wikipedia.org',wiki_adapter)


def get_id(name):         # func for catching ids
  # Construct the API URL
  api_url = f'https://en.wikipedia.org/w/api.php?action=query&titles={name}&prop=pageprops&format=json'

# Make the API request and convert the response to JSON
  try:
    response = session1.get(api_url).json()
  except ConnectionError as ce:
    print(ce)

# Try to Extract the page ID from the JSON response
  try:
    page_id = str(next(iter(response['query']['pages'].values()))['pageprops']['wikibase_item'])
    #print(page_id)
    return page_id

  except Exception as e:
    return None

#########################################################################################################
#########################################################################################################

wdata_adapter = HTTPAdapter(max_retries=99)
session2 = requests.Session()
session2.mount('https://www.wikidata.org',wdata_adapter)
session2.headers.update({'User-Agent': 'Custom user agent'})

def get_name_info(page_id):
    url = f"https://www.wikidata.org/wiki/{page_id}"

    try:
      response1 = session2.get(url)
      trash = response1.content
    except ConnectionError as er:
      print('>>>',er)


    soup = BeautifulSoup(response1.text, "html.parser")
    my_blocks = soup.find('ul',class_='wikibase-sitelinklistview-listview').findAll(recursive=False)    #all elements we need from some name`s page.

    my_name = soup.find('span',class_ = 'wikibase-title-label').text                                    #Element with a main name`s translate
    my_description = soup.find('span',class_ = 'wikibase-descriptionview-text').text                    #Element with description of a current name

    for my_block in my_blocks:
        s1=my_block.findAll({'span':True})                  #All elements we need of every additional translate name

        my_language = s1[1].get('title')
        my_translate = s1[2].text
        my_lang = s1[3].get('lang')

        result = (my_name,page_id,my_description,my_language,my_lang,my_translate)
        result_tuples.append(result)    # appending tuple to a manager(list)

#########################################################################################################
#########################################################################################################


if __name__ == '__main__':

    manager = multiprocessing.Manager()
    result_tuples = manager.list()      # Manager for collecting all tuples from multiprocessing in one data structure.

    with multiprocessing.Pool(4) as pool: # Multiprocessing

      all_ids = pool.map(get_id ,tqdm(names))      # Mapping all names we found with get_id func in order to find and save all IDs.
      valid_ids = filter(lambda x:x is not None, tqdm(all_ids)) # Removing elements with value "None".
      pool.map(get_name_info, tqdm(list(valid_ids)))

    result_tuples = list(result_tuples) #Convert manager object to
    results_df = pd.DataFrame(result_tuples, columns=['Label','WikiData ID','English Description','Language','Wiki Short Lang','Entry'])
    results_df.to_csv('family_names.csv', index=False, encoding='utf-8-sig')


100%|██████████| 70646/70646 [1:37:12<00:00, 12.11it/s]
